<center>
    <span style="font-size: 1.0em; color:black"><b>
        June 27$^\mathrm{st}$, 2023
    </b></span>
    <br>
    <span style="font-size: 2.5em; color:black"><b>
        Work Notebook
    </b></span>
    <br>
    <span style="font-size: 0.75em;"><i>
        Eduardo J. Alvarez
    </i></span> 
    <br>
    <span style="font-size: 0.75em;"><i>
        The content of this notebook is intellectual property of <a href="https://www.whisper.aero/">Whisper Aero</a>. <br>
        No use or sharing of this information is allowed without written consent.
    </i></span> 
</center>

# DuctTAPE &mdash; Neumann Solver

## Constant Doublet Panel Definition

An axisymmetric doublet panel is defined by its two end points $\mathbf{p}_1$ and $\mathbf{p}_2$ and its strength $\mu$. 

If $\mu$ is positive, the doublet panel is equivalent two vortex rings, one at each end point, as shown below, where a positive vortex ring strength $\Gamma$ is defined through the right-hand rule as a circulation coming out of the plane:

<img src="img/2ddoubletpanel00.png" alt="pic here" width="30%">

The normal $\hat{\mathbf{n}}$ of the panel is defined as shown above.

The velocity induced by such panels is computed as follows:

```julia
import DuctTAPE as dt


"""
Returns the x-component of velocity induced by the vortex ring of strength 
`Gammasrc`  and position `Xsrc` at a target point `Xtrg`.
"""
function vx(Gammasrc, Xsrc, Xtrg)
    
    #= NOTE:
        * "j-th panel" = source
        * "affecting point" = source
        * "i-th" panel = target
        * "affected point" = target
    =#
    
    x, r, m, rj = dt.calculate_xrm(Xsrc, Xtrg)
    
    return Gammasrc * dt.get_vx_ring_vortex_off_body(x, r, rj, m)
    
end

"""
Returns the r-component of velocity induced by the vortex ring of strength 
`Gammasrc`  and position `Xsrc` at a target point `Xtrg`.
"""
function vr(Gammasrc, Xsrc, Xtrg)
    
    x, r, m, rj = dt.calculate_xrm(Xsrc, Xtrg)
    
    return Gammasrc * dt.get_vr_ring_vortex_off_body(x, r, rj, m)
    
end

"""
Calculate the velocity induced by axisymmetric doublet panel of strength `mu`
and end points `p1` and `p2` on target `Xtrg`. The velocity components are
added to the vector `out`. 
"""
function U_axisymdoublet!(out, mu, p1, p2, Xtrg)
    
    # NOTE: I have flipped the Gammas to be Γ1=-μ and Γ2=μ to account for the fact
    #       that Judd's implementation defined a positive Γ as a ring with circulation 
    #       going into the $xr$-plane
    
    # Velocity induced by p1 ring
    out[1] += vx(-mu, p1, Xtrg)
    out[2] += vr(-mu, p1, Xtrg)
    
    # Velocity induced by p2 ring
    out[1] += vx(mu, p2, Xtrg)
    out[2] += vr(mu, p2, Xtrg)
    
    return out
end

function U_axisymdoublet(mu::T1, 
                            p1::AbstractVector{T2}, p2::AbstractVector{T3}, 
                            Xtrg::AbstractVector{T4}) where {T1, T2, T3, T4}
    T = promote_type(T1, T2, T3, T4)
    out = zeros(T, 2)
    
    U_axisymdoublet!(out, mu, p1, p2, Xtrg)
    
    return out
end
```

NOTE
* $u_r$ is NaN at $r=0$
* $u_x$ is 0 at $x=x_0$ and $r=R$, instead of $\infty$
* A positive $\Gamma$ defines a ring with circulation going into the $xr$-plane (meaning, a ring traveling in the $-x$ direction)

As a reference here is the velocity field induced around an isolated constant doublet panel:
<img src="img/axisymdoublet00.png" alt="pic here" width="100%">

<img src="img/axisymdoublet01.png" alt="pic here" width="100%">

## No-Flow-Through Boundary Condition

The velocity $\mathbf{u}_i$ induced by a doublet panel of strength $\mu$ at an arbitrary position $\mathbf{x}$ is given by

\begin{align*}
    \mathbf{u}_i \left( \mathbf{x} \right) = \mu \mathbf{g}(\mathbf{x})
,\end{align*}

where the geometric function $\mathbf{g}$ is computed from the end points of the panel alone.
Given an axisymmetric body discretized into $N$ doublet panels, the velocity induced by the body at any point in space is then computed as

\begin{align*}
        \mathbf{u} \left( \mathbf{x} \right)
    = 
        \sum\limits_{i=1}^N \mathbf{u}_i \left( \mathbf{x} \right) 
    = 
        \sum\limits_{i=1}^N \mu_i \mathbf{g}_i(\mathbf{x})
.\end{align*}

In order to impose the no-flow-through condition, we require that the velocity induced by the body cancels out the normal component of the freestream velocity along the body:

\begin{align*}
    \mathbf{u} \left( \mathbf{x}_j \right) \cdot \hat{\mathbf{n}}_j
    + 
    \mathbf{u}_\infty \cdot \hat{\mathbf{n}}_j
    =
    0
\end{align*}

where $\mathbf{x}_j$ and $\hat{\mathbf{n}}_j$ are the control point and normal associated to the $j$-th target panel.

The no-flow-through condition can then be rewritten as


\begin{align*}
        \sum\limits_{i=1}^N \mu_i \mathbf{g}_i(\mathbf{x}_j) \cdot \hat{\mathbf{n}}_j
    =
        - \mathbf{u}_\infty \cdot \hat{\mathbf{n}}_j
,\end{align*}

which poses a system of $N$ equations (one for each $\mathbf{x}_j$) and $N$ unknowns (each $\mu_i$).
We rewrite this system of equations as

\begin{align*}
        G \boldsymbol\mu = \mathbf{b}
,\end{align*}

where $G$ is an $N\times N$ matrix containing the geometric information ($G_{ji} \equiv \mathbf{g}_i(\mathbf{x}_j) \cdot \hat{\mathbf{n}}_j$), $\boldsymbol\mu$ is the vector of doublet strengths, and $\mathbf{b}$ is the vector of boundary conditions ($b_j = - \mathbf{u}_\infty \cdot \hat{\mathbf{n}}_j$).

The geometric matrix $G$ is built as follows:

```julia
function G_axisymdoublet!(G, nodes::AbstractMatrix, panels, 
                                controlpoints::AbstractMatrix, normals::AbstractMatrix)
    
    # Iterate over boundary condition equations
    for (j, (xj, nj)) in enumerate(zip(eachcol(controlpoints), eachcol(normals)))
        
        # Iterate over panels inducing velocity
        for (i, panel) in enumerate(panels)
                
            # Fetch panel end points
            p1 = view(nodes, :, panel[1])
            p2 = view(nodes, :, panel[2])
                
            # Velocity induced by i-th panel with unitary strength at j-th control point, 𝐠𝑖(𝐱𝑗)
            U = U_axisymdoublet(1.0, p1, p2, xj)
                
            # Build geometric matrix, 𝐺𝑗𝑖 = 𝐠𝑖(𝐱𝑗)⋅𝐧̂𝑗
            G[j, i] += dot(U, nj)
            
        end
    end
        
    return G
end
    
function G_axisymdoublet(nodes::AbstractMatrix{T1}, panels, 
                            controlpoints::AbstractMatrix{T2}, normals::AbstractMatrix{T3}
                        ) where {T1, T2, T3}
    T = promote_type(T1, T2, T3)
    N = length(panels)
    G = zeros(T, N, N)
        
    G_axisymdoublet!(G, nodes, panels, controlpoints, normals)
        
    return G
end
```



Here is a function that puts together the right-hand-side of the boundary conditions (vector $\mathbf{b}$):
```julia
function bc_noflowthrough!(b, nodes::AbstractMatrix, panels, 
                                controlpoints::AbstractMatrix, normals::AbstractMatrix, 
                                Us::AbstractMatrix)
    
    # Iterate over boundary condition equations
    for (j, (Uj, nj)) in enumerate(zip(eachcol(Us), eachcol(normals)))
        
        b[j] -= dot(Uj, nj)
        
    end
        
    return b
end

function bc_noflowthrough(nodes::AbstractMatrix, panels, 
                            controlpoints, normals::AbstractMatrix{T1}, Us::AbstractMatrix{T2}
                        ) where {T1, T2}
    T = promote_type(T1, T2)
    N = length(panels)
    b = zeros(T, N)
        
    bc_noflowthrough!(b, nodes, panels, controlpoints, normals, Us)
        
    return b
end
```

For completeness, here is a function that evaluates the velocity induced by a paneled body at collection of target position `Xtargets`:

```julia
"""
Calculate the velocity induced by a paneled body on each target `Xtargets`
"""
function Ubody!(Us::AbstractMatrix, Xtargets::AbstractMatrix, nodes, panels, mus)
    
    # Iterate over targets
    for (j, (U, Xtrg)) in enumerate(zip(eachcol(Us), eachcol(Xtargets)))
        
        # Iterate over panels
        for (panel, mu) in zip(panels, mus)

            # Fetch panel end points
            p1 = view(nodes, :, panel[1])
            p2 = view(nodes, :, panel[2])

            # Add velocity induced by this panel on the j-th target
            U_axisymdoublet!(U, mu, p1, p2, Xtrg)
            
        end
    end
    
    return Us
end

function Ubody(Xtargets::AbstractMatrix{T1}, 
                nodes::AbstractMatrix{T2}, panels, mus::AbstractVector{T3}) where {T1, T2, T3}
    
    T = promote_type(T1, T2, T3)
    Us = zeros(T, 2, size(Xtargets, 2))
    
    Ubody!(Us, Xtargets, nodes, panels, mus)
    
    return Us
end
```

For comparison, here is a paneled body solving the boundary conditions both with FLOWPanel (3D panel method) and our axisymmetric method. Notice that there is a lot of numerical noise that creates discrepancies in the contours. This noise is because the system of equations is ill conditioned as we will see next.

<table>
    <tr>
        <th><center>3D Panel Method</center></th>
        <th><center>Axisymmetric Method</center></th>
    </tr>
    <tr>
        <td>
            <img src="img/ducthill-3d-comp00-u00.png" alt="pic here" width="100%">
        </td>
        <td>
            <img src="img/ducthill-axisym-comp00-u00.png" alt="pic here" width="100%">
        </td>
    </tr>
    <tr>
        <td>
            <img src="img/ducthill-3d-comp00-cp00.png" alt="pic here" width="100%">
        </td>
        <td>
            <img src="img/ducthill-axisym-comp00-cp00.png" alt="pic here" width="100%">
        <td>
    </tr>
</table>

## Least-Squares Solver


It is well known that a purely doublet (or vortex ring) solver encounters
difficulties when the geometry is closed (*i.e.*, watertight).
The flow field around a body is given by the vortex filaments that make the
edges of the panel, and the
strength of each filament is simply the difference between adjacent panels.
Thus, in the absence of an open edge (like in a watertight geometry), the
strengths of the doublet elements become irrelevant, and the problem
is purely determined by the difference in strength between adjacent panels.
This leads to an overdetermined problem where one of the original degrees of
freedom (panels strengths) has become redundant.

Let $n$ be the number of panels. The problem is well defined for an open
geometry formulating the solver as

\begin{align*}
    G \boldsymbol\mu = \mathbf{b}
,\end{align*}

where $\mathbf{b} \in \mathbb{R}^{n}$ is the normal of the freestream condition
at each panel that needs to be canceled ("no-flow-through" boundary
condition), $G \in \mathbb{R}^{n\times n}$ contains the geometric
information of the panels and $\boldsymbol\mu \in \mathbb{R}^{n}$ is the
strengths of doublet panel.
However, for a watertight geometry, $G$ is no longer full rank and the
problem is ill-conditioned.
Due to numerical roundoff, the system of equations can still be inverted
but the numerical solution ends up giving panel strengths $\mu$ that are in the order of $10^{16}$ and large numerical
noise.

In order to circumvent this issue, we can transform the original
problem into a least-squares problem as follows.
Since one of the panel strengths is redundant in a watertight geometry,
we can simply pick an arbitrary panel and prescribe its strength to any arbitrary value.
Then, $G$ is reduced to a $n\times n-1$ matrix $G_\mathrm{red}$ and $\boldsymbol\mu$ is reduced to a
vector $\boldsymbol\mu_\mathrm{red}$ of length $n-1$, while $\mathbf{b}$ is still a vector of length $n$.
To formulate the least-squares problem, we substract the velocity $b_p$
induced by the "prescribed" panel  to the right-hand side,

\begin{align*}
    G_\mathrm{red} \boldsymbol\mu_\mathrm{red} = \mathbf{b} - \mathbf{b}_p
,\end{align*}

and we solve the problem as

\begin{align*}
    \boldsymbol\mu_\mathrm{red} = \left( G_\mathrm{red}^{t} G_\mathrm{red} \right)^{-1} G_\mathrm{red}^{t} \left( \mathbf{b} - \mathbf{b}_p \right)
,\end{align*}

where the superscript $t$ denotes the transposed matrix.

For simplicity, we write the least-squares solution as a linear solver of the form

\begin{align*}
    \boldsymbol\mu_\mathrm{red} 
    = 
    G_\mathrm{ls}^{-1} \mathbf{b}_\mathrm{ls}
,\end{align*}

where $G_\mathrm{ls} \equiv G_\mathrm{red}^{t} G_\mathrm{red}$ and $\mathbf{b}_\mathrm{ls} \equiv G_\mathrm{red}^{t} \left( \mathbf{b} - \mathbf{b}_p \right)$.

Here is a function that takes in both the original geometric matrix $G$ and RHS $\mathbf{b}$ and returns their least-squares version $G_\mathrm{ls}$ and $\mathbf{b}_\mathrm{ls}$:
```julia
"""
Given the original system of equations G*mu = b, it converts it into its 
equivalent least-squares problem by prescribing the strengths of the
panels under `prescribedpanels` and returning Gls and bls.

NOTE: b is modified in place to become b-bp. 
NOTES 2: Gred is an auxiliary matrix used to build Gls and bls.
"""
function G_leastsquares!(Gred, Gls, bls, G, b, prescribedpanels)
    
    #=
        prescribedpanels = [ 
                                [prescribed_panel_index1, prescribed_strength1],
                                [prescribed_panel_index2, prescribed_strength2],
                                ...
                            ]
    =#
    
    # Total number of panels
    n = length(b)
    
    # Number of prescribed panels
    npres = length(prescribedpanels)
    
    # Error cases
    @assert size(G, 1)==n && size(G, 2)==n ""*
        "Invalid $(size(G, 1))x$(size(G, 2)) matrix G; expected $(n)x$(n)"
    @assert size(Gred, 1)==n && size(Gred, 2)==n-npres ""*
        "Invalid $(size(Gred, 1))x$(size(Gred, 2)) matrix Gred; expected $(n)x$(n-npres)"
    @assert size(Gls, 1)==n-npres && size(Gls, 2)==n-npres ""*
        "Invalid $(size(Gls, 1))x$(size(Gls, 2)) matrix Gls; expected $(n-npres)x$(n-npres)"

    @assert length(b)==n "Invalid b length $(length(b)); expected $(n)"
    @assert length(bls)==n-npres "Invalid bls length $(length(bls)); expected $(n-npres)"

    # Sort prescribed elements by index
    sort!(prescribedpanels, by = x -> x[1])
    
    # Move influence of prescribed panels to right-hand side
    for (paneli, strength) in prescribedpanels
        for i in 1:length(b)
            b[i] -= strength*G[i, paneli]
        end
    end
    
    # Reduce G: copy G into Gred without the prescribed panels
    prev_paneli = 0
    for (i, (paneli, strength)) in enumerate(prescribedpanels)

        Gred[:, (prev_paneli+2-i):(paneli-i)] .= view(G, :, (prev_paneli+1):(paneli-1))

        if i==length(prescribedpanels) && paneli!=size(G, 2)
            Gred[:, (paneli-i+1):end] .= view(G, :, paneli+1:size(G, 2))
        end

        prev_paneli = paneli
    end
    
    tGred = transpose(Gred)

    # Store Gred'*(b - bp) under bls
    LA.mul!(bls, tGred, b)

    # Store Gred'*Gred under Gls
    LA.mul!(Gls, tGred, Gred)
    
    return Gls, bls
end

function G_leastsquares(G::AbstractMatrix{T1}, b::AbstractVector{T2}, 
                        prescribedpanels::AbstractArray{Tuple{Int, T3}}) where {T1, T2, T3}

    T = promote_type(T1, T2, T3)

    n = length(b)
    npres = length(prescribedpanels)

    Gred = zeros(T, n, n-npres)
    Gls = zeros(T, n-npres, n-npres)
    bls = zeros(T, n-npres)

    G_leastsquares!(Gred, Gls, bls, G, b, prescribedpanels)
    
    return Gls, bls
end
```

This function takes in the reduced strength vector $\boldsymbol\mu_\mathrm{ls}$ and returns the full vector of strengths $\boldsymbol\mu$:
```julia
"""
Converts the reduced vector of strengths to the full vector of strengths
"""
function mured2mu!(mu, mured, prescribedpanels)
    
    # Total number of panels
    n = length(mu)
    
    # Number of prescribed panels
    npres = length(prescribedpanels)
    
    # Case of no prescrbied panels
    if npres==0
        
        mu .= mured
        return mu
        
    end
    
    prev_paneli = 0
    
    # Iterate over prescribed panels building the full vector
    for (i, (paneli, strength)) in enumerate(prescribedpanels)
        mu[(prev_paneli+1):(paneli-1)] .= view(mured, (prev_paneli+2-i):(paneli-i))
        mu[paneli] = strength

        if i==npres && paneli!=n
            mu[paneli+1:end] .= view(mured, (paneli-i+1):length(mured))
        end

        prev_paneli = paneli
    end
    
    return mu
end

function mured2mu(mured::AbstractVector{T1}, 
                    prescribedpanels::AbstractArray{Tuple{Int, T2}}
                    ) where {T1, T2}

    T = promote_type(T1, T2)
    
    n = length(mured) + length(prescribedpanels)
    mu = zeros(T, n)

    mured2mu!(mu, mured, prescribedpanels)
    
    return mu
end
```

Now we see that the numerical noise has dissapeared and the axisymmetric solution matches the full 3D solution on point:

<table>
    <tr>
        <th><center>3D Panel Method</center></th>
        <th><center>Axisymmetric Method</center></th>
    </tr>
    <tr>
        <td>
            <img src="img/ducthill-3d-comp01-u00.png" alt="pic here" width="100%">
        </td>
        <td>
            <img src="img/ducthill-axisym-comp01-u00.png" alt="pic here" width="100%">
        </td>
    </tr>
    <tr>
        <td>
            <img src="img/ducthill-3d-comp01-cp00.png" alt="pic here" width="100%">
        </td>
        <td>
            <img src="img/ducthill-axisym-comp01-cp00.png" alt="pic here" width="100%">
        <td>
    </tr>
</table>

## Simple Linear Kutta Condition

While the Kutta condition requires that the pressure at the trailing edge is the same at both sides of an airfoil, this poses a non-linear equation:

\begin{align*}
        p_\mathrm{upper} & = p_\mathrm{lower}
    \\ \Leftrightarrow \qquad
        p_0 + \frac{\rho \Vert \mathbf{u}_\mathrm{upper} \Vert^2}{2}
        & =
        p_0 + \frac{\rho \Vert \mathbf{u}_\mathrm{lower} \Vert^2}{2}
\end{align*}

\begin{align*}
    \Rightarrow \qquad
        \boxed{
            \mathbf{u}_\mathrm{upper} \cdot \mathbf{u}_\mathrm{upper}
            =
            \mathbf{u}_\mathrm{lower} \cdot \mathbf{u}_\mathrm{lower}
        }
\end{align*}

Instead of requiring the velocity magnitude to be equal, we can approximate the Kutta condition requiring only the tangential component to be equal, resulting in a linear equation:


\begin{align*}
        \mathbf{u}_\mathrm{upper} \cdot \hat{\mathbf{t}}_\mathrm{upper}
        =
        \mathbf{u}_\mathrm{lower} \cdot \hat{\mathbf{t}}_\mathrm{lower}
\end{align*}

This introduces one extra equation. Luckily, the Kutta condition also includes one degree of freedom: assuming steady conditions, the starting vortex creates a semi-infinite vortex ring (or a constant-strength doublet panel) that stems from the TE out to infinite:

<img src="img/doublet-kutta00.png" alt="pic here" width="50%">

The strength of this wake semi-infinite panel, $\mu_w$, is unkown and it falls out from the Kutta condition equation along with the boundary conditions. Zooming into the trailing edge, we have a junction of three doublet panels:
<img src="img/doublet-kutta02.png" alt="pic here" width="70%">

Since an axisymmetric doublet panel of constant strength is equivalent to having two vortex rings at the end points, this junction is then equivalent to a vortex ring of strength $\Gamma_\mathrm{TE}$:

\begin{align*}
    \Gamma_\mathrm{TE}
    \equiv 
    \mu_\mathrm{wake} + \mu_\mathrm{lower} - \mu_\mathrm{upper}
.\end{align*}

One can then further approximate the linear Kutta condition assuming that equating tangential velocities (meaning, no flow goes around the TE), is roughly close to having no circulation at the trailing edge, $\Gamma_\mathrm{TE} = 0$. Hence, we further simplify the Kutta condition equation to be

\begin{align*}
    \mu_\mathrm{wake} = \mu_\mathrm{upper} - \mu_\mathrm{lower}
.\end{align*}

Adding the velocity induced by the wake panel, the boundary conditions now becomes:

\begin{align*}
        \sum\limits_{i=1}^N \mu_i \mathbf{g}_i(\mathbf{x}_j) \cdot \hat{\mathbf{n}}_j
        +
        \sum\limits_{k=1}^M \mu_k^w \mathbf{g}_k^w(\mathbf{x}_j) \cdot \hat{\mathbf{n}}_j
    =
        - \mathbf{u}_\infty \cdot \hat{\mathbf{n}}_j
,\end{align*}

here we have generalized to the case that we have $M$ trailing edges (as in the case of solving for multiple lifting bodies), and $\mathbf{g}_k^w$ is the geometric function of the $k$-th wake panel. Using the simplified Kutta condition, $\mu_\mathrm{wake} = \mu_\mathrm{upper} - \mu_\mathrm{lower}$, this equation then becomes:

\begin{align*}
        \sum\limits_{i=1}^N \mu_i \mathbf{g}_i(\mathbf{x}_j) \cdot \hat{\mathbf{n}}_j
        +
        \sum\limits_{k=1}^M \left( \mu_k^\mathrm{upper} - \mu_k^\mathrm{lower} \right) \mathbf{g}_k^w(\mathbf{x}_j) \cdot \hat{\mathbf{n}}_j
    =
        - \mathbf{u}_\infty \cdot \hat{\mathbf{n}}_j
.\end{align*}

The system of equations can be written in matrix form as

\begin{align*}
        G \boldsymbol\mu + G_\mathrm{wake} \boldsymbol\mu = \mathbf{b}
\end{align*}

or 

\begin{align*}
        G^* \boldsymbol\mu = \mathbf{b}
,\end{align*}

where $G_\mathrm{wake}$ contains the geometry of the wakes and $G^* \equiv G + G_\mathrm{wake}$. Hence, applying this linearized and simplified Kutta condition is as simple as modifying the entries of the original $G$ matrix to account for the wake.

Here is the vorticity field induced by the body. Notice that there is only vorticity (vortex rings) at the edge of the panels, as expected, while there is no vortex at the trailing edge, which was canceled by the wake:
<img src="img/axysim-duct-hill03-w01.png" alt="pic here" width="100%">

Here is a function that modifies the original geometric matrix $G$ (before the least-squares modification) to include the Kutta condition (it converts $G$ into $G^*$):

```julia
function G_kutta_linearsimple!(G, nodes::AbstractMatrix, panels, 
                                controlpoints::AbstractMatrix, normals::AbstractMatrix,
                                wakes::AbstractVector{T}; tol=1e1*eps()) where {T<:AbstractVector{Int}}
    
    #=
        wake1 = [upper_panel_index, lower_panel_index]
        wake2 = ...
        ...
        wakes = [wake1, wake2, ...]
    =#
    
    prev_pte = nothing
    
    # Iterate over boundary condition equations
    for (j, (xj, nj)) in enumerate(zip(eachcol(controlpoints), eachcol(normals)))
        
        # Iterate over wake panels inducing velocity
        for wake in wakes
            
            # Iterate over TE sides
            for (i, paneli) in enumerate(wake)
                # i is used to determine side (i==1 => upper, i=2 => lower)
                
                # Fetch panel end points
                p1 = view(nodes, :, panels[paneli][1])
                p2 = view(nodes, :, panels[paneli][2])
                
                # Position of TE vortex
                pte = i%2==1 ? p2 : p1
                
                # Unitary strength of this component of TE vortex
                mute = i%2==1 ? 1 : -1
                
                # Error case: TE points between upper and lower side are not coincident
                if j==1 && !isnothing(prev_pte) && !isnothing(tol)
                    @assert norm(pte-prev_pte) <= tol "TE points are not coincident: $(prev_pte) and $(pte)"
                end
                
                # Velocity induced by wake panel with unitary strength at j-th control point, 𝐠𝑖(𝐱𝑗)
                U = [vx(-mute, pte, xj), vr(-mute, pte, xj)]
                
                # Add term to geometric matrix, 𝐺𝑗𝑖 = 𝐠𝑖(𝐱𝑗)⋅𝐧̂𝑗
                G[j, paneli] += dot(U, nj)
                
                prev_pte = pte
            end
            
        end
        
    end
        
    return G
end
    
function G_kutta_linearsimple!(nodes::AbstractMatrix{T1}, panels, 
                                controlpoints::AbstractMatrix{T2}, normals::AbstractMatrix{T3},
                                args...; optargs...
                                ) where {T1, T2, T3}
    T = promote_type(T1, T2, T3)
    N = length(panels)
    G = zeros(T, N, N)
        
    G_kutta_linearsimple!(G, nodes, panels, controlpoints, normals, args...; optargs...)
        
    return G
end

```

Here is a function that evaluates the velocity induced by the wake:

```julia


"""
Calculate the velocity induced by the wake of a paneled body on each target `Xtargets`
"""
function Uwake!(Us::AbstractMatrix, Xtargets::AbstractMatrix, nodes, panels, mus, wakes; tol=1e1*eps())
    
    prev_pte = nothing
    
    # Iterate over targets
    for (j, (U, Xtrg)) in enumerate(zip(eachcol(Us), eachcol(Xtargets)))
        
        
        # Iterate over wake panels inducing velocity
        for wake in wakes
            
            # Iterate over TE sides
            for (i, paneli) in enumerate(wake)
                # i is used to determine side (i==1 => upper, i=2 => lower)
                
                # Fetch panel end points
                p1 = view(nodes, :, panels[paneli][1])
                p2 = view(nodes, :, panels[paneli][2])
                
                # Position of TE vortex
                pte = i%2==1 ? p2 : p1
                
                # Unitary strength of this component of TE vortex
                mute = i%2==1 ? 1 : -1
                mute *= mus[paneli]
                
                # Error case: TE points between upper and lower side are not coincident
                if j==1 && !isnothing(prev_pte) && !isnothing(tol)
                    @assert norm(pte-prev_pte) <= tol "TE points are not coincident: $(prev_pte) and $(pte)"
                end
                
                # Velocity induced by wake panel
                U[1] += vx(-mute, pte, Xtrg)
                U[2] += vr(-mute, pte, Xtrg)
                
                prev_pte = pte
                
            end
            
        end
        
    end
    
    return Us
end


function Uwake(Xtargets::AbstractMatrix{T1}, 
                nodes::AbstractMatrix{T2}, panels, mus::AbstractVector{T3}, wakes; 
                optargs...) where {T1, T2, T3}
    
    T = promote_type(T1, T2, T3)
    Us = zeros(T, 2, size(Xtargets, 2))
    
    Uwake!(Us, Xtargets, nodes, panels, mus, wakes; optargs...)
    
    return Us
end
```

Solution of least-squares solver with the Kutta condition:
<table>
    <tr>
        <th><center>3D Panel Method</center></th>
        <th><center>Axisymmetric Method</center></th>
    </tr>
    <tr>
        <td>
            <img src="img/ducthill-3d-comp02-u00.png" alt="pic here" width="100%">
        </td>
        <td>
            <img src="img/ducthill-axisym-comp02-u00.png" alt="pic here" width="100%">
        </td>
    </tr>
    <tr>
        <td>
            <img src="img/ducthill-3d-comp02-cp00.png" alt="pic here" width="100%">
        </td>
        <td>
            <img src="img/ducthill-axisym-comp02-cp00.png" alt="pic here" width="100%">
        <td>
    </tr>
</table>

## Surface Velocity ($-\frac{\nabla \mu \left( \mathbf{x} \right)}{2}$ Term)

It is a long derivation, but the discontinuity of the potential field at the interface between the open field and surfaces leads to deriving the following expression for the surface velocity with boundary elements:

\begin{align*}
        \mathbf{u} \left( \mathbf{x} \right)
    = 
        \mathbf{u}_\infty
        +
        \mathbf{u}_\mathrm{body} \left( \mathbf{x} \right)
        +
        \mathbf{u}_\mathrm{wake} \left( \mathbf{x} \right)
        -
        \frac{\nabla \mu \left( \mathbf{x} \right)}{2}
,\end{align*}

where $\mathbf{x}$ is on the surface of a panel and $\mu \left( \mathbf{x} \right)$ is the doublet strength of that panel. 

In order to calculate that surface gradient, we use the [Green-Gauss method](https://ntrs.nasa.gov/api/citations/20140011550/downloads/20140011550.pdf). For a three-dimensional surface the gradient is computed as 

\begin{align*}
        \nabla \mu 
    = 
        \frac{1}{A} \sum\limits_f^{N_\mathrm{faces}} \overline{\mu}_f \hat{\mathbf{n}}_f \ell_f
,\end{align*}

where $A$ is the area of the element, $\ell_f$ is the length of the face, $\hat{\mathbf{n}}_f$ is the normal to the face, and $\overline{\mu}_f$ is the average strength over the face. 

In an axisymmetric scheme, we have $A = \pi \ell (r_1 + r_2)$ where $\ell = \Vert \mathbf{p}_2 - \mathbf{p}_1 \Vert$, $\ell_f = 2\pi r_f$, and the gradient is computed as

\begin{align*}
        \nabla \mu 
    = 
        \frac{1}{\pi \ell (r_1 + r_2)} \sum\limits_f^{N_\mathrm{endpoints}} \overline{\mu}_f \hat{\mathbf{n}}_f 2\pi r_f
,\end{align*}


\begin{align*}
    \Rightarrow
        \nabla \mu 
    = 
        \frac{1}{\ell \frac{r_1 + r_2}{2}} \sum\limits_f^{N_\mathrm{endpoints}} \overline{\mu}_f \hat{\mathbf{n}}_f r_f
,\end{align*}

Here we will use the inverse-distance-weighted face averaging:

\begin{align*}
        \overline{\mu}_f 
    = 
        \frac{
            \sum\limits_i^n \frac{\mu_i}{ \Vert \mathbf{d}_i \Vert^2}
        }{
            \sum\limits_i^n \frac{1}{ \Vert \mathbf{d}_i \Vert^2}
        }
,\end{align*}

where the sums iterate over the panels in contact with the corresponding face and $\mathbf{d}_f$ is the distance from the center of each panel to the face.

> **NOTE:** The gradient-term velocity is purely tangential to the panel. Hence, this term has no effect on the no-flow-through boundary condition.

Here is a function calculating the surface velocity using the Green-Gauss method:
```julia
function Ugradmu!(U::AbstractMatrix, nodes::AbstractMatrix{T1}, panels, 
                            mus::AbstractVector{T2}, wakes
                        ) where {T1, T2}
    
    T = promote_type(T1, T2)
    
    # Iterate over panels
    for (i, panel) in enumerate(panels)
        
        # Fetch panel end points
        p1 = view(nodes, :, panel[1])
        p2 = view(nodes, :, panel[2])
        
        # Panel length
        ell = norm(p2 - p1)
        
        # Panel midpoint
        pmidself = (p1 + p2)/2
        
        # Panel strength
        mu = mus[i]
        
        # Agglomerate ∇𝜇 here
        gradmu = zeros(T, 2)
        
        # Iterate over faces (end points)
        for (fi, f) in enumerate(panel)
            
            # Fetch this end point
            pf = view(nodes, :, f)
            
            # Panel index of this neighbor
            neighbori = fi%2==1 ? i-1 : i+1
            neighbori = neighbori==(length(panels)+1) ? 1 : neighbori==0 ? length(panels) : neighbori
            
            # Fetch midpoint of this neighbor
            pf1 = view(nodes, :, panels[neighbori][1])
            pf2 = view(nodes, :, panels[neighbori][2])
            pmidf = (pf1 + pf2) / 2
            
            # Fetch strength of this neighbor
            muf = mus[neighbori]
            
            # Calculate direction vectors
            df_to_self = pmidself - pf
            df_to_f = pmidf - pf
            magdf_to_self = norm(df_to_self)
            magdf_to_f = norm(df_to_f)
            
            # Calculate \bar{𝜇}_𝑓            
            num = mu/magdf_to_self + muf/magdf_to_f
            den = 1/magdf_to_self + 1/magdf_to_f
            
            barmuf = num/den
            
            # Calculate normal vector
            # NOTE: is this normal pointing in the right direction?
            hatnf = df_to_self/magdf_to_self
            
            # Add ∑𝜇𝐧̂𝑟
            gradmu += barmuf * hatnf * pf[2]
            
        end
        
        # ∇𝜇 = 1/(ℓ(r1 + r2)/2) ∑𝜇𝐧̂𝑟
        gradmu /= ell * (p1[2] + p2[2])/2
        
        # Add -∇𝜇/2 surface velocity term
        U[:, i] -= gradmu/2
        
    end
    
    return U
    
end


function Ugradmu(nodes::AbstractMatrix{T1}, panels, mus::AbstractVector{T2}, args...) where {T1, T2}
    
    T = promote_type(T1, T2)
    Us = zeros(T, 2, length(panels))
    
    Ugradmu!(Us, nodes, panels, mus, args...)
    
    return Us
end
```

Unfortunately, my implementation in the axisymmetric code gets a different answer than what we had implemented in FLOWPanel, and spite of much debugging I still don't know what's wrong:
<img src="img/duct-greengauss00.png" alt="pic here" width="100%">

<img src="img/duct-greengauss01.png" alt="pic here" width="50%">

Noticing that the slopes of the doublet strengths $\mu$ between FLOWPanel and the axisymmetric method match fairly well (see plots above), instead of using the Green-Gauss method I ended up finite differencing $\mu$ as follows:

```julia
function Ugradmu!(U::AbstractMatrix, nodes::AbstractMatrix{T1}, panels, 
                            mus::AbstractVector{T2}, wakes
                        ) where {T1, T2}

    T = promote_type(T1, T2)

    # Iterate over panels
    for (i, panel) in enumerate(panels)

        # Fetch panel end points
        p1 = view(nodes, :, panel[1])
        p2 = view(nodes, :, panel[2])

        # Panel length
        ell = norm(p2 - p1)
        
        # Tangent to this panel
        dhat = (p2-p1)/ell

        # Normal to this panel
        nhat = cross([0, 0, 1], vcat(dhat, 0))
        nhat = nhat[1:2]

        # Panel midpoint
        pmidself = (p1 + p2)/2

        # Panel strength
        mu = mus[i]

        # Agglomerate ∇𝜇 here
        gradmu = zeros(T, 2)

        wtot = zero(T)

        # Iterate over faces (end points)
        for (fi, f) in enumerate(panel)

            # Fetch this end point
            pf = view(nodes, :, f)

            # Panel index of this neighbor
            neighbori = fi%2==1 ? i-1 : i+1
            neighbori = neighbori==(length(panels)+1) ? 1 : neighbori==0 ? length(panels) : neighbori

            # Fetch midpoint of this neighbor
            pf1 = view(nodes, :, panels[neighbori][1])
            pf2 = view(nodes, :, panels[neighbori][2])
            pmidf = (pf1 + pf2) / 2

            # Fetch strength of this neighbor
            muf = mus[neighbori]

            # Weight of this neighbor
            ellf = norm(pf2 - pf1)
            wf = ellf

            # Gradient defined as (𝜇 - 𝜇f) / (x - xf)
            thisgrad = (mu - muf) ./ (pmidself - pmidf)

            # Build the gradient as a weighted average between neighbors
            gradmu += wf * thisgrad

            wtot += wf

        end

        # Finish the weighted average
        gradmu ./= wtot

        # Now weight each derivative by the anti-normal component
        gradmu .*= (1 .- nhat.^2)

        # Get rid of spurious numerical noise when the deltas are very small
        if norm(gradmu) > 1e6
            gradmu *= 0
        end

        # Add -∇𝜇/2 surface velocity term
        U[:, i] -= gradmu/2

    end

    return U

end
```

Results with the finite difference:
<img src="img/duct-finitediff00.png" alt="pic here" width="100%">

<img src="img/duct-finitediff01.png" alt="pic here" width="50%">

## Validation Case

Here we verify the implementation by comparing the results to experimental measurements on a duct.

### Load Environment

In [2]:
using Pkg

Pkg.status()

Project DuctTAPE v0.1.0
Status `~/code-development/DuctTAPE.jl/Project.toml`
  [e1828068] CCBlade
  [fb58ae67] FLOWFoil
  [6cb5d3fb] FLOWMath
  [6a86dc24] FiniteDiff
  [f6369f11] ForwardDiff
  [e7cbb90b] ImplicitAD
  [42fd0dbc] IterativeSolvers
  [d3d80556] LineSearches
  [2774e3e8] NLsolve
  [276daf66] SpecialFunctions
  [37e2e46d] LinearAlgebra
  [de0858da] Printf
  [10745b16] Statistics
  [8dfed614] Test


In [23]:
# envpath = "environments/ducttape001"


#= CREATE THE ENVIRONMENT
using Pkg
Pkg.activate(envpath)

Pkg.add("PyPlot")
Pkg.add(url="https://github.com/byuflowlab/GeometricTools.jl", rev="Master")
Pkg.add(url="https://github.com/byuflowlab/FLOWPanel.jl", rev="UDeltaGamma")

Pkg.add(url="https://github.com/byuflowlab/FLOWFoil.jl", rev="dev")
Pkg.add(url="https://github.com/byuflowlab/DuctTAPE.jl", rev="eec345f03eeb6f29426c50fb9bc875d20851b71b")


Pkg.develop(path="/home/edoalvar/Dropbox/WhisperAero/FLOWCodes/FLOWExaFMM")
Pkg.add(url="https://github.com/byuflowlab/FLOWVPM.jl")

Pkg.status()
=#

using Pkg
# Pkg.activate(envpath)

import GeometricTools as gt
using DuctTAPE 
dt = DuctTAPE

import FLOWPanel as pnl

import LinearAlgebra as LA

import PyPlot as plt
import PyPlot: @L_str

# Fonts
plt.rc("font", family="Times New Roman")            # Text font
plt.rc("mathtext", fontset="stix")                  # Math font
plt.rc("font", size=12)          # controls default text sizes
plt.rc("axes", titlesize=12)     # fontsize of the axes title
plt.rc("axes", labelsize=14)     # fontsize of the x and y labels
plt.rc("xtick", labelsize=12)    # fontsize of the tick labels
plt.rc("ytick", labelsize=12)    # fontsize of the tick labels
plt.rc("legend", fontsize=12)    # legend fontsize
plt.rc("figure", titlesize=16)   # fontsize of the figure title

Pkg.status()

# Auxiliary Functions
dot(A, B) = sum(a*b for (a,b) in zip(A, B))
norm(A) = sqrt(mapreduce(x->x^2, +, A))
function cross!(out, A, B)
    out[1] = A[2]*B[3] - A[3]*B[2]
    out[2] = A[3]*B[1] - A[1]*B[3]
    out[3] = A[1]*B[2] - A[2]*B[1]
    return out
end
function cross(A::AbstractVector{T1}, B::AbstractVector{T2}) where {T1, T2}
    out = zeros(promote_type(T1, T2), 3)
    return cross!(out, A, B)
end
mean(xs) = sum(xs)/length(xs)

[ Info: Precompiling DuctTAPE [ad8e49fd-fab7-444e-af4a-0daba3b8bf11]
ERROR: LoadError: ArgumentError: Package FLOWMath [6cb5d3fb-0fe8-4cc2-bd89-9fe0b19a99d3] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.

Stacktrace:
 [1] _require(pkg::Base.PkgId, env::String)
   @ Base ./loading.jl:1774
 [2] _require_prelocked(uuidkey::Base.PkgId, env::String)
   @ Base ./loading.jl:1660
 [3] macro expansion
   @ ./loading.jl:1648 [inlined]
 [4] macro expansion
   @ ./lock.jl:267 [inlined]
 [5] require(into::Module, mod::Symbol)
   @ Base ./loading.jl:1611
 [6] include
   @ ./Base.jl:457 [inlined]
 [7] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::Nothing)
   @ Base ./loading.jl:2045
 [8] top-level scope
   @ stdin:3
in expression starting at /home/juddmehr/code-development/DuctTAPE.

LoadError: Failed to precompile DuctTAPE [ad8e49fd-fab7-444e-af4a-0daba3b8bf11] to "/home/juddmehr/.julia/compiled/v1.9/DuctTAPE/jl_HNIeiy".

### Axisymmetric Code Implementation

In [4]:
"""
Returns the x-component of velocity induced by the vortex ring of strength 
`Gammasrc`  and position `Xsrc` at a target point `Xtrg`.
"""
function vx(Gammasrc, Xsrc, Xtrg)
    
    #= NOTE:
        * "j-th panel" = source
        * "affecting point" = source
        * "i-th" panel = target
        * "affected point" = target
    =#
    
    x, r, m, rj = dt.calculate_xrm(Xsrc, Xtrg)
    
    return Gammasrc * dt.get_vx_ring_vortex_off_body(x, r, rj, m)
    
end

"""
Returns the r-component of velocity induced by the vortex ring of strength 
`Gammasrc`  and position `Xsrc` at a target point `Xtrg`.
"""
function vr(Gammasrc, Xsrc, Xtrg)
    
    x, r, m, rj = dt.calculate_xrm(Xsrc, Xtrg)
    
    return Gammasrc * dt.get_vr_ring_vortex_off_body(x, r, rj, m)
    
end

"""
Calculate the velocity induced by axisymmetric doublet panel of strength `mu`
and end points `p1` and `p2` on target `Xtrg`. The velocity components are
added to the vector `out`. 
"""
function U_axisymdoublet!(out, mu, p1, p2, Xtrg)
    
    # NOTE: I have flipped the Gammas to be Γ1=-μ and Γ2=μ to account for the fact
    #       that Judd's implementation defined a positive Γ as a ring with circulation 
    #       going into the $xr$-plane
    
    # Velocity induced by p1 ring
    out[1] += vx(-mu, p1, Xtrg)
    out[2] += vr(-mu, p1, Xtrg)
    
    # Velocity induced by p2 ring
    out[1] += vx(mu, p2, Xtrg)
    out[2] += vr(mu, p2, Xtrg)
    
    return out
end

function U_axisymdoublet(mu::T1, 
                            p1::AbstractVector{T2}, p2::AbstractVector{T3}, 
                            Xtrg::AbstractVector{T4}) where {T1, T2, T3, T4}
    T = promote_type(T1, T2, T3, T4)
    out = zeros(T, 2)
    
    U_axisymdoublet!(out, mu, p1, p2, Xtrg)
    
    return out
end

function G_axisymdoublet!(G, nodes::AbstractMatrix, panels, 
                                controlpoints::AbstractMatrix, normals::AbstractMatrix)
    
    # Iterate over boundary condition equations
    for (j, (xj, nj)) in enumerate(zip(eachcol(controlpoints), eachcol(normals)))
        
        # Iterate over panels inducing velocity
        for (i, panel) in enumerate(panels)
                
            # Fetch panel end points
            p1 = view(nodes, :, panel[1])
            p2 = view(nodes, :, panel[2])
                
            # Velocity induced by i-th panel with unitary strength at j-th control point, 𝐠𝑖(𝐱𝑗)
            U = U_axisymdoublet(1.0, p1, p2, xj)
                
            # Build geometric matrix, 𝐺𝑗𝑖 = 𝐠𝑖(𝐱𝑗)⋅𝐧̂𝑗
            G[j, i] += dot(U, nj)
            
        end
    end
        
    return G
end
    
function G_axisymdoublet(nodes::AbstractMatrix{T1}, panels, 
                            controlpoints::AbstractMatrix{T2}, normals::AbstractMatrix{T3}
                        ) where {T1, T2, T3}
    T = promote_type(T1, T2, T3)
    N = length(panels)
    G = zeros(T, N, N)
        
    G_axisymdoublet!(G, nodes, panels, controlpoints, normals)
        
    return G
end

function bc_noflowthrough!(b, nodes::AbstractMatrix, panels, 
                                controlpoints::AbstractMatrix, normals::AbstractMatrix, 
                                Us::AbstractMatrix)
    
    # Iterate over boundary condition equations
    for (j, (Uj, nj)) in enumerate(zip(eachcol(Us), eachcol(normals)))
        
        b[j] -= dot(Uj, nj)
        
    end
        
    return b
end

function bc_noflowthrough(nodes::AbstractMatrix, panels, 
                            controlpoints, normals::AbstractMatrix{T1}, Us::AbstractMatrix{T2}
                        ) where {T1, T2}
    T = promote_type(T1, T2)
    N = length(panels)
    b = zeros(T, N)
        
    bc_noflowthrough!(b, nodes, panels, controlpoints, normals, Us)
        
    return b
end

function calc_normals!(normals, nodes, panels)
    
    for (n, panel) in zip(eachcol(normals), panels)
        
        # Fetch panel end points
        p1 = view(nodes, :, panel[1])
        p2 = view(nodes, :, panel[2])
        
        rmag = sqrt((p2[1] - p1[1])^2 + (p2[2] - p1[2])^2)
        
        # nhat = khat x (p2-p1)/mag(p2-p1)
        n[1] = -(p2[2] - p1[2])/rmag
        n[2] = (p2[1] - p1[1])/rmag
    end
    
    return normals
end

function calc_normals(nodes::AbstractMatrix{T}, panels) where {T}
    N = length(panels)
    normals = zeros(T, 2, N)
        
    calc_normals!(normals, nodes, panels)
        
    return normals
end


function calc_controlpoints!(controlpoints, nodes, panels)
    
    for (cp, panel) in zip(eachcol(controlpoints), panels)
        
        # Fetch panel end points
        p1 = view(nodes, :, panel[1])
        p2 = view(nodes, :, panel[2])
        
        cp[1] = (p1[1] + p2[1])/2
        cp[2] = (p1[2] + p2[2])/2
    end
    
    return controlpoints
end

function calc_controlpoints(nodes::AbstractMatrix{T}, panels) where {T}
    N = length(panels)
    controlpoints = zeros(T, 2, N)
        
    calc_controlpoints!(controlpoints, nodes, panels)
        
    return controlpoints
end

function save_vtk(prefix, save_path, nodes, panels, mus, controlpoints, normals; 
                                                            Us=nothing, optargs...)
    
    points = [[p[1], p[2], 0] for p in eachcol(nodes)]
    cells = [panel .- 1 for panel in panels]
    
    cell_data = [
                    Dict(
                        "field_name" => "mu",
                        "field_type" => "scalar",
                        "field_data" => mus
                    )
                ]
    
    
    point_data = [
                    Dict(
                        "field_name" => "normal",
                        "field_type" => "vector",
                        "field_data" => [[n[1], n[2], 0] for n in eachcol(normals)]
                    )
                ]
    
    if !isnothing(Us)
        push!(point_data, Dict(
                                    "field_name" => "U",
                                    "field_type" => "vector",
                                    "field_data" => [[U[1], U[2], 0] for U in eachcol(Us)]
                                )
            )
    end
    
    cp_points = [[p[1], p[2], 0] for p in eachcol(controlpoints)]
    
    str = ""
    str *= gt.generateVTK(prefix*"_panel", points; cells=cells, cell_data=cell_data, path=save_path, optargs...)
    str *= gt.generateVTK(prefix*"_cp", cp_points; point_data=point_data, path=save_path, optargs...)
    
    return str
end

function G_kutta_linearsimple!(G, nodes::AbstractMatrix, panels, 
                                controlpoints::AbstractMatrix, normals::AbstractMatrix,
                                wakes::AbstractVector{T}; tol=1e1*eps()) where {T<:AbstractVector{Int}}
    
    #=
        wake1 = [upper_panel_index, lower_panel_index]
        wake2 = ...
        ...
        wakes = [wake1, wake2, ...]
    =#
    
    prev_pte = nothing
    
    # Iterate over boundary condition equations
    for (j, (xj, nj)) in enumerate(zip(eachcol(controlpoints), eachcol(normals)))
        
        # Iterate over wake panels inducing velocity
        for wake in wakes
            
            # Iterate over TE sides
            for (i, paneli) in enumerate(wake)
                # i is used to determine side (i==1 => upper, i=2 => lower)
                
                # Fetch panel end points
                p1 = view(nodes, :, panels[paneli][1])
                p2 = view(nodes, :, panels[paneli][2])
                
                # Position of TE vortex
                pte = i%2==1 ? p2 : p1
                
                # Unitary strength of this component of TE vortex
                mute = i%2==1 ? 1 : -1
                
                # Error case: TE points between upper and lower side are not coincident
                if j==1 && !isnothing(prev_pte) && !isnothing(tol)
                    @assert norm(pte-prev_pte) <= tol "TE points are not coincident: $(prev_pte) and $(pte)"
                end
                
                # Velocity induced by wake panel with unitary strength at j-th control point, 𝐠𝑖(𝐱𝑗)
                U = [vx(-mute, pte, xj), vr(-mute, pte, xj)]
                
                # Add term to geometric matrix, 𝐺𝑗𝑖 = 𝐠𝑖(𝐱𝑗)⋅𝐧̂𝑗
                G[j, paneli] += dot(U, nj)
                
                prev_pte = pte
            end
            
        end
        
    end
        
    return G
end
    
function G_kutta_linearsimple!(nodes::AbstractMatrix{T1}, panels, 
                                controlpoints::AbstractMatrix{T2}, normals::AbstractMatrix{T3},
                                args...; optargs...
                                ) where {T1, T2, T3}
    T = promote_type(T1, T2, T3)
    N = length(panels)
    G = zeros(T, N, N)
        
    G_kutta_linearsimple!(G, nodes, panels, controlpoints, normals, args...; optargs...)
        
    return G
end

"""
Given the original system of equations G*mu = b, it converts it into its 
equivalent least-squares problem by prescribing the strengths of the
panels under `prescribedpanels` and returning Gls and bls.

NOTE: b is modified in place to become b-bp. 
NOTES 2: Gred is an auxiliary matrix used to build Gls and bls.
"""
function G_leastsquares!(Gred, Gls, bls, G, b, prescribedpanels)
    
    #=
        prescribedpanels = [ 
                                [prescribed_panel_index1, prescribed_strength1],
                                [prescribed_panel_index2, prescribed_strength2],
                                ...
                            ]
    =#
    
    # Total number of panels
    n = length(b)
    
    # Number of prescribed panels
    npres = length(prescribedpanels)
    
    # Error cases
    @assert size(G, 1)==n && size(G, 2)==n ""*
        "Invalid $(size(G, 1))x$(size(G, 2)) matrix G; expected $(n)x$(n)"
    @assert size(Gred, 1)==n && size(Gred, 2)==n-npres ""*
        "Invalid $(size(Gred, 1))x$(size(Gred, 2)) matrix Gred; expected $(n)x$(n-npres)"
    @assert size(Gls, 1)==n-npres && size(Gls, 2)==n-npres ""*
        "Invalid $(size(Gls, 1))x$(size(Gls, 2)) matrix Gls; expected $(n-npres)x$(n-npres)"

    @assert length(b)==n "Invalid b length $(length(b)); expected $(n)"
    @assert length(bls)==n-npres "Invalid bls length $(length(bls)); expected $(n-npres)"

    # Sort prescribed elements by index
    sort!(prescribedpanels, by = x -> x[1])
    
    # Move influence of prescribed panels to right-hand side
    for (paneli, strength) in prescribedpanels
        for i in 1:length(b)
            b[i] -= strength*G[i, paneli]
        end
    end
    
    # Reduce G: copy G into Gred without the prescribed panels
    prev_paneli = 0
    for (i, (paneli, strength)) in enumerate(prescribedpanels)

        Gred[:, (prev_paneli+2-i):(paneli-i)] .= view(G, :, (prev_paneli+1):(paneli-1))

        if i==length(prescribedpanels) && paneli!=size(G, 2)
            Gred[:, (paneli-i+1):end] .= view(G, :, paneli+1:size(G, 2))
        end

        prev_paneli = paneli
    end
    
    tGred = transpose(Gred)

    # Store Gred'*(b - bp) under bls
    LA.mul!(bls, tGred, b)

    # Store Gred'*Gred under Gls
    LA.mul!(Gls, tGred, Gred)
    
    return Gls, bls
end

function G_leastsquares(G::AbstractMatrix{T1}, b::AbstractVector{T2}, 
                        prescribedpanels::AbstractArray{Tuple{Int, T3}}) where {T1, T2, T3}

    T = promote_type(T1, T2, T3)

    n = length(b)
    npres = length(prescribedpanels)

    Gred = zeros(T, n, n-npres)
    Gls = zeros(T, n-npres, n-npres)
    bls = zeros(T, n-npres)

    G_leastsquares!(Gred, Gls, bls, G, b, prescribedpanels)
    
    return Gls, bls
end

"""
Converts the reduced vector of strengths to the full vector of strengths
"""
function mured2mu!(mu, mured, prescribedpanels)
    
    # Total number of panels
    n = length(mu)
    
    # Number of prescribed panels
    npres = length(prescribedpanels)
    
    # Case of no prescrbied panels
    if npres==0
        
        mu .= mured
        return mu
        
    end
    
    prev_paneli = 0
    
    # Iterate over prescribed panels building the full vector
    for (i, (paneli, strength)) in enumerate(prescribedpanels)
        mu[(prev_paneli+1):(paneli-1)] .= view(mured, (prev_paneli+2-i):(paneli-i))
        mu[paneli] = strength

        if i==npres && paneli!=n
            mu[paneli+1:end] .= view(mured, (paneli-i+1):length(mured))
        end

        prev_paneli = paneli
    end
    
    return mu
end

function mured2mu(mured::AbstractVector{T1}, 
                    prescribedpanels::AbstractArray{Tuple{Int, T2}}
                    ) where {T1, T2}

    T = promote_type(T1, T2)
    
    n = length(mured) + length(prescribedpanels)
    mu = zeros(T, n)

    mured2mu!(mu, mured, prescribedpanels)
    
    return mu
end



"""
Calculate the velocity induced by a paneled body on each target `Xtargets`
"""
function Ubody!(Us::AbstractMatrix, Xtargets::AbstractMatrix, nodes, panels, mus)
    
    # Iterate over targets
    for (j, (U, Xtrg)) in enumerate(zip(eachcol(Us), eachcol(Xtargets)))
        
        # Iterate over panels
        for (panel, mu) in zip(panels, mus)

            # Fetch panel end points
            p1 = view(nodes, :, panel[1])
            p2 = view(nodes, :, panel[2])

            # Add velocity induced by this panel on the j-th target
            U_axisymdoublet!(U, mu, p1, p2, Xtrg)
            
        end
    end
    
    return Us
end

function Ubody(Xtargets::AbstractMatrix{T1}, 
                nodes::AbstractMatrix{T2}, panels, mus::AbstractVector{T3}) where {T1, T2, T3}
    
    T = promote_type(T1, T2, T3)
    Us = zeros(T, 2, size(Xtargets, 2))
    
    Ubody!(Us, Xtargets, nodes, panels, mus)
    
    return Us
end


"""
Calculate the velocity induced by the wake of a paneled body on each target `Xtargets`
"""
function Uwake!(Us::AbstractMatrix, Xtargets::AbstractMatrix, nodes, panels, mus, wakes; tol=1e1*eps())
    
    prev_pte = nothing
    
    # Iterate over targets
    for (j, (U, Xtrg)) in enumerate(zip(eachcol(Us), eachcol(Xtargets)))
        
        
        # Iterate over wake panels inducing velocity
        for wake in wakes
            
            # Iterate over TE sides
            for (i, paneli) in enumerate(wake)
                # i is used to determine side (i==1 => upper, i=2 => lower)
                
                # Fetch panel end points
                p1 = view(nodes, :, panels[paneli][1])
                p2 = view(nodes, :, panels[paneli][2])
                
                # Position of TE vortex
                pte = i%2==1 ? p2 : p1
                
                # Unitary strength of this component of TE vortex
                mute = i%2==1 ? 1 : -1
                mute *= mus[paneli]
                
                # Error case: TE points between upper and lower side are not coincident
                if j==1 && !isnothing(prev_pte) && !isnothing(tol)
                    @assert norm(pte-prev_pte) <= tol "TE points are not coincident: $(prev_pte) and $(pte)"
                end
                
                # Velocity induced by wake panel
                U[1] += vx(-mute, pte, Xtrg)
                U[2] += vr(-mute, pte, Xtrg)
                
                prev_pte = pte
                
            end
            
        end
        
    end
    
    return Us
end


function Uwake(Xtargets::AbstractMatrix{T1}, 
                nodes::AbstractMatrix{T2}, panels, mus::AbstractVector{T3}, wakes; 
                optargs...) where {T1, T2, T3}
    
    T = promote_type(T1, T2, T3)
    Us = zeros(T, 2, size(Xtargets, 2))
    
    Uwake!(Us, Xtargets, nodes, panels, mus, wakes; optargs...)
    
    return Us
end


function Ugradmu!(U::AbstractMatrix, nodes::AbstractMatrix{T1}, panels, 
                            mus::AbstractVector{T2}, wakes
                        ) where {T1, T2}

    T = promote_type(T1, T2)

    # Iterate over panels
    for (i, panel) in enumerate(panels)

        # Fetch panel end points
        p1 = view(nodes, :, panel[1])
        p2 = view(nodes, :, panel[2])

        # Panel length
        ell = norm(p2 - p1)
        
        # Tangent to this panel
        dhat = (p2-p1)/ell

        # Normal to this panel
        nhat = cross([0, 0, 1], vcat(dhat, 0))
        nhat = nhat[1:2]

        # Panel midpoint
        pmidself = (p1 + p2)/2

        # Panel strength
        mu = mus[i]

        # Agglomerate ∇𝜇 here
        gradmu = zeros(T, 2)

        wtot = zero(T)

        # Iterate over faces (end points)
        for (fi, f) in enumerate(panel)

            # Fetch this end point
            pf = view(nodes, :, f)

            # Panel index of this neighbor
            neighbori = fi%2==1 ? i-1 : i+1
            neighbori = neighbori==(length(panels)+1) ? 1 : neighbori==0 ? length(panels) : neighbori

            # Fetch midpoint of this neighbor
            pf1 = view(nodes, :, panels[neighbori][1])
            pf2 = view(nodes, :, panels[neighbori][2])
            pmidf = (pf1 + pf2) / 2

            # Fetch strength of this neighbor
            muf = mus[neighbori]

            # Weight of this neighbor
            ellf = norm(pf2 - pf1)
            wf = ellf

            # Gradient defined as (𝜇 - 𝜇f) / (x - xf)
            thisgrad = (mu - muf) ./ (pmidself - pmidf)

            # Build the gradient as a weighted average between neighbors
            gradmu += wf * thisgrad

            wtot += wf

        end

        # Finish the weighted average
        gradmu ./= wtot

        # Now weight each derivative by the anti-normal component
        gradmu .*= (1 .- nhat.^2)

        # Get rid of spurious numerical noise when the deltas are very small
        if norm(gradmu) > 1e6
            gradmu *= 0
        end

        # Add -∇𝜇/2 surface velocity term
        U[:, i] -= gradmu/2

    end

    return U

end

function Ugradmu(nodes::AbstractMatrix{T1}, panels, mus::AbstractVector{T2}, args...) where {T1, T2}
    
    T = promote_type(T1, T2)
    Us = zeros(T, 2, length(panels))
    
    Ugradmu!(Us, nodes, panels, mus, args...)
    
    return Us
end

Ugradmu (generic function with 1 method)

### Run Simulation

In [5]:
#=##############################################################################
# DESCRIPTION
    Fan duct replicating the experiment reported by V. P. Hill (1978), "A
    Surface Vorticity Theory for Propeller Ducts and Turbofan Engine Cowls in
    Non-Axisymmetric Incompressible Flow." The same experiment is also
    discussed in Sections 4.4 and 6.3.1 of Lewis, R. (1991), "Vortex Element
    Methods for Fluid Dynamic Analysis of Engineering Systems."
=###############################################################################

import FLOWPanel as pnl
import CSV
import DataFrames: DataFrame


save_path       = "/home/juddmehr/code-development/DuctTAPE.jl/examples/new_panels/"     # Where to save outputs
run_name        = "duct"                    # Name of this run

verbose         = true                      # Wheter to verbose
v_lvl           = 0                         # Verbose indentation level
paraview        = !true                      # Wheter to call ParaView


# ----------------- SIMULATION PARAMETERS --------------------------------------
magVinf         = 30.0                      # (m/s) freestream velocity
rho             = 1.225                     # (kg/m^3) air density


# ----------------- GEOMETRY DESCRIPTION ---------------------------------------
# Read duct contour (table in figure 7.4 of Lewis 1991)
filename        = joinpath(pnl.examples_path, "data", "naca662015.csv")
contour         = CSV.read(filename, DataFrame)

aspectratio     = 0.6                       # Duct trailing edge aspect ratio l/d
d               = 2*0.835                   # (m) duct diameter


# ----------------- SOLVER PARAMETERS ------------------------------------------
# Discretization

# NOTE: NDIVS is the number of divisions (panels) in each dimension. This can be
#       either an integer, or an array of tuples as shown below

n_rfl           = 20                        # This controls the number of chordwise panels

NDIVS_rfl_up = [                            # Discretization of airfoil upper surface
            # 0 to 0.25 of the airfoil has `n_rfl` panels at a geometric expansion of 10 that is not central
                (0.25, n_rfl,   10.0, false),
            # 0.25 to 0.75 of the airfoil has `n_rfl` panels evenly spaced
                (0.50, n_rfl,    1.0, true),
            # 0.75 to 1.00 of the airfoil has `n_rfl` panels at a geometric expansion of 0.1 that is not central
                (0.25, n_rfl,    0.1, false)]

NDIVS_rfl_lo = NDIVS_rfl_up                 # Discretization of airfoil lower surface

# NOTE: A geometric expansion of 10 that is not central means that the last
#       panel is 10 times larger than the first panel. If central, the
#       middle panel is 10 times larger than the peripheral panels.


# ----------------- GENERATE AIRFOIL CONTOUR -----------------------------------
# Re-discretize the contour of the body of revolution according to NDIVS
xs, ys = gt.rediscretize_airfoil(contour[:, 1], contour[:, 2],
                                        NDIVS_rfl_up, NDIVS_rfl_lo;
                                        verify_spline=false)

# Make sure that the contour is closed
# NOTE: Closing the contour makes the problem ill-conditioned since
#       one of the degrees of freedom dissappears
ys[end] = ys[1]

# Scale contour by duct length
xs *= d*aspectratio
ys *= d*aspectratio

# Move contour to the radial position
ys .+= d/2

# Revert points to make the normals point outwards
reverse!(xs)
reverse!(ys)

# Verify discretization
fig = plt.figure(figsize=[12, 5])
ax = fig.gca()

ax.plot(contour[!, 1], contour[!, 2] .+ d/2, ".k", linewidth=1, markersize=8, clip_on=false, label="Raw")
ax.plot(xs, ys, ".-", linewidth=1, markersize=8, clip_on=false, label="Panels")
ax.set_aspect("equal")

ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)

vtks = save_path*"/"                        # String with VTK output files



# ----------------- GENERATE PANELED BODY --------------------------------------
# Collect points that make the contour of the body of revolution
nodes = collect(transpose(hcat(xs, ys)))
panels = [[i, i+1] for i in 1:size(nodes, 2)-1]

npanels = length(panels)

controlpoints = calc_controlpoints(nodes, panels)
normals = calc_normals(nodes, panels)

ax.plot(controlpoints[1, :], controlpoints[2, :], "xr", markersize=4, label="Control points")

# Define trailing edge
wakes = [[npanels, 1]]
# wakes = Vector{Int}[]

# Plot wake for verification
for wake in wakes
    # Iterate over TE sides
    for (i, paneli) in enumerate(wake)
        # i is used to determine side (i==1 => upper, i=2 => lower)

        # Fetch panel end points
        p1 = view(nodes, :, panels[paneli][1])
        p2 = view(nodes, :, panels[paneli][2])

        # Position of TE vortex
        pte = i%2==1 ? p2 : p1
        
        ax.plot([pte[1], pte[1] + 0.1*d*aspectratio], pte[2]*ones(2), "-c", label="Wake"^(i==1))
    end
    
end

ax.legend(loc="best", fontsize=9)


# ----------------- SOLVER -----------------------------------------------------
# Freestream at each control point
Uinf = magVinf*[1, 0]
Uinfs = repeat(Uinf, 1, npanels)



LoadError: UndefVarError: `plt` not defined

In [6]:
# Geometric matrix (left-hand side)
G = G_axisymdoublet(nodes, panels, controlpoints, normals)



LoadError: UndefVarError: `nodes` not defined

In [7]:
# Add linear simple Kutta condition
G_kutta_linearsimple!(G, nodes, panels, controlpoints, normals, wakes; tol=1e-3)



LoadError: UndefVarError: `G` not defined

In [8]:
# Boundary condition vector (right-hand side)
b = bc_noflowthrough(nodes, panels, controlpoints, normals, Uinfs)



LoadError: UndefVarError: `nodes` not defined

In [9]:
# # Doublet strengths through direct linear solver
# mus = G\b

# Prescribe the strength of an arbitrary panel to be zero
prescribedpanels = [(1, 0.0)]

# Convert the ill-conditioned linear system of equations to a least-squares problem
Gls, bls = G_leastsquares(G, b, prescribedpanels)
display(Gls)
display(bls)


LoadError: UndefVarError: `G` not defined

In [10]:
# Solve the least-squares problem with a linear solver returning the reduced strengths
mureds = Gls\bls



LoadError: UndefVarError: `Gls` not defined

In [11]:
# Build the full vector of strengths
mus = mured2mu(mureds, prescribedpanels)




LoadError: UndefVarError: `mureds` not defined

In [12]:
# ---------------- GENERATE FLUID DOMAIN ---------------------------------------
# Bounds of grid
Pmax = d*[aspectratio*1.5, 0.5*1.35, 0] # Upper bound
Pmin = d*[-aspectratio*0.35, eps(), 0]            # Lower bound

# Grid discretization
dx         = 0.005*d*aspectratio                            # Cell size
# dx         = 0.0005*d*aspectratio
# dx         = 0.001*d*aspectratio
dr         = dx

NDIVS = ceil.(Int, (Pmax .- Pmin) ./ [dx, dr, 1]) # Divisions in each dimension


# Generate grid
@time grid  = gt.Grid(Pmin, Pmax, NDIVS) # Grid

if verbose; println("\t"^(v_lvl+1)*"Grid size:\t\t$(NDIVS)"); end;
if verbose; println("\t"^(v_lvl+1)*"Number of nodes :\t$(grid.nnodes)"); end;

# Targets where to probe the velocity
targets = grid.nodes
ntargets = size(targets, 2)


  0.376480 seconds (1.72 M allocations: 74.399 MiB, 4.14% gc time, 78.72% compilation time)
	Grid size:		[370, 225, 0]
	Number of nodes :	83846


83846

In [13]:
# Velocity at each target
Us = repeat(Uinf, 1, ntargets) 
Us += Ubody(targets, nodes, panels, mus)
Us += Uwake(targets, nodes, panels, mus, wakes; tol=1e-3)
Us = [[U[1], U[2], 0] for U in eachcol(Us)]

# Pressure field
Cps = 1 .- (norm.(Us)/magVinf).^2

# Save fields
gt.add_field(grid, "U", "vector", Us, "node")
gt.add_field(grid, "Cp", "scalar", Cps, "node")

# Output fluid domain
@time vtks = gt.save(grid, run_name; path=save_path, format="vtk")

LoadError: UndefVarError: `Uinf` not defined

In [14]:
# Output paneled body
Us_cp = Uinfs 


LoadError: UndefVarError: `Uinfs` not defined

In [15]:
Us_b = Ubody(controlpoints, nodes, panels, mus) 
Us_cp += Us_b
display(Us_b')

LoadError: UndefVarError: `controlpoints` not defined

In [16]:
Us_w = Uwake(controlpoints, nodes, panels, mus, wakes; tol=1e-3)
Us_cp += Us_w
display(Us_w')

LoadError: UndefVarError: `controlpoints` not defined

In [17]:
#check vdotn
vn = dot.(eachcol(Us_cp), eachcol(normals))

LoadError: UndefVarError: `dot` not defined

In [18]:
normals

LoadError: UndefVarError: `normals` not defined

In [19]:
vtks *= save_vtk(run_name, save_path, nodes, panels, mus, controlpoints, normals; Us=Us_cp)

# Call ParaView
if paraview
    str = save_path*"/"*vtks
    run(`paraview --data=$(str)`)
end

LoadError: UndefVarError: `vtks` not defined

In [20]:
Us_gm = Ugradmu(nodes, panels, mus, wakes)
Us_cp += Us_gm
display(Us_gm')

LoadError: UndefVarError: `nodes` not defined

In [21]:
# Surface pressure
Cps = 1 .- (norm.(eachcol(Us_cp))/magVinf).^2
display(Cps)

xs = controlpoints[1, :] / (d*aspectratio)
xs .-= minimum(xs)

fig = plt.figure(figsize=[7, 5*0.8]*2/3)
ax = fig.gca()

# Plot experimental surface pressure (figures 4.8 and 6.4 in Lewis 1991)
Cps_exp = [
                0.006611570247933851 0.6680814940577251
                0.02479338842975201 -0.021222410865874286
                0.04958677685950391 -0.12308998302207141
                0.08264462809917339 -0.16893039049236003
                0.12892561983471057 -0.2351443123938881
                0.18429752066115695 -0.2538200339558573
                0.24214876033057842 -0.2877758913412565
                0.3115702479338843 -0.3081494057724957
                0.3851239669421487 -0.31833616298811535
                0.45206611570247934 -0.328522920203735
                0.5297520661157025 -0.3336162988115449
                0.6024793388429752 -0.33022071307300527
                0.6760330578512396 -0.26910016977928664
                0.7479338842975207 -0.21307300509337868
                0.8223140495867767 0.055178268251273255
                0.8801652892561982 0.13157894736842102
                0.0049586776859503745 0.4864176570458403
                0.020661157024793292 -0.09762308998302205
                0.04876033057851231 -0.24533106960950757
                0.08512396694214858 -0.3183361629881155
                0.12892561983471068 -0.42869269949066197
                0.177685950413223 -0.46774193548387083
                0.2380165289256197 -0.5492359932088285
                0.30661157024793373 -0.5933786078098472
                0.3752066115702479 -0.6222410865874364
                0.44793388429752046 -0.6392190152801358
                0.5264462809917354 -0.6528013582342954
                0.5991735537190083 -0.6171477079796265
                0.6735537190082643 -0.5356536502546689
                0.7446280991735537 -0.3777589134125636
                0.8223140495867769 -0.04668930390492365
                0.8801652892561984 0.07555178268251284
            ]

ax.plot(Cps_exp[:, 1], Cps_exp[:, 2], "ok", markersize=5, label="Experimental")

# Plot predicted
ax.plot(xs, Cps, "-", color="cyan", linewidth=2.0, alpha=0.9, label="Panel")

# Beautify the plot
xlims = [-0.1, 1.1]
ylims = [1.0, -2.5]
ax.set_xlim(xlims)
ax.set_xticks(0:0.25:1.0)
ax.set_ylim(ylims)
ax.set_yticks(ylims[1]:-1.0:ylims[2])

ax.set_xlabel(L"x/c")
ax.set_ylabel(L"Pressure $C_p$")
ax.legend(loc="best", frameon=false, fontsize=10)

ax.spines["right"].set_visible(false)
ax.spines["top"].set_visible(false)

LoadError: UndefVarError: `norm` not defined